In [ ]:
using Pkg
Pkg.activate("/Users/bbrener1/haxx/RNAForecasterPaperCode/")

In [ ]:
include("../src/trainRNAForecasterV2.jl")

In [ ]:
include("../src/recursivePredictionsV2.jl")

In [ ]:
using DelimitedFiles
t0 = transpose(readdlm("sc_velo_t0_emb.tsv"))
t1 = transpose(readdlm("sc_velo_t1_pred_emb.tsv"))
t0 = Float32.(t0)
t1 = Float32.(t1)


In [ ]:
trainedModel,losses = trainRNAForecaster(t0, t1, hiddenLayerNodes = 1000,
 batchsize = 200, learningRate = 0.0001, checkStability = false, useGPU = false, nEpochs = 200)


In [ ]:
using JLD2
outModel = cpu(trainedModel)
save_object("pancNODEEmbedded.jld2", Flux.state(outModel))


In [ ]:
# # Let's load a trained model
# using Flux: loadmodel!
# using JLD2
# function loadForecaster(fileName::String, inputNodes::Int, hiddenLayerNodes::Int)
#     #recreate neural network structure
#     nn = Chain(Dense(inputNodes, hiddenLayerNodes, relu),
#                Dense(hiddenLayerNodes, inputNodes))
#     model = NeuralODE(nn, (0.0f0, 1.0f0), Tsit5(),
#                        save_everystep = false,
#                        reltol = 1e-3, abstol = 1e-3,
#                        save_start = false)
#     #load parameters into the model
#     model = loadmodel!(model, load_object(fileName))
#     return model
# end



In [ ]:
# trained = loadForecaster("pancNeuralODEResult.jld2", 1352, 6000)

In [ ]:
# target = Float32.(splicedSub)
# size(target)

In [ ]:
include("../src/recursivePredictionsV2.jl")

In [ ]:
futures = predictCellFutures(trainedModel,t0,6,damping=1.0f0)

In [ ]:
# losses

In [ ]:
writedlm("./trajectories_t1.tsv",futures[:,:,1])
writedlm("./trajectories_t2.tsv",futures[:,:,2])
writedlm("./trajectories_t3.tsv",futures[:,:,3])
writedlm("./trajectories_t4.tsv",futures[:,:,4])
writedlm("./trajectories_t5.tsv",futures[:,:,5])
writedlm("./trajectories_t6.tsv",futures[:,:,6])

In [ ]:
futures[:,1,1]

In [ ]:
futures[:,1,2]

In [ ]:
t0[:,1]

In [ ]:
t1[:,1]

In [ ]:
futures[:,1,:]

In [ ]:
# residual = t1.-futures[:,:,1]
residual = t1.-t0
sq_res = residual.^2
mean(sq_res)